In [1]:
import os
import pyro
import torch
import pickle
import time

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import pyro.distributions as dist

from tqdm.auto import trange
from pyro.nn import PyroModule, PyroSample, PyroParam
from pyro.infer import MCMC, NUTS, SVI, Trace_ELBO, Predictive
from pyro.distributions import constraints
from pyro.infer.autoguide import AutoDiagonalNormal

os.chdir("../../")

from src.dgp_rff.deep_layer import DeepGP, DeepGPNoBias, DeepEnsembleGP
# from src.dgp_rff.get_variable import getvalue

In [2]:
import torch
torch.cuda.is_available()
# trouble shoot see this link:https://stackoverflow.com/questions/77068908/how-to-install-pytorch-with-cuda-support-on-windows-11-cuda-12-no-matching

True

In [3]:
start = time.perf_counter()
torch.cuda.current_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)
# cuda = torch.device('cuda') 

In [4]:
var1= torch.FloatTensor([1.0,2.0,3.0]).cuda()

In [5]:
var1.device

device(type='cuda', index=0)

In [6]:
torch.cuda.memory_allocated()

512

  # Next step

0. Get familiar with the coding structure
1. CPU -> GPU
2. Last step without bias
3. Figure out how they train
4. How to access the posterior mean and std/scale from the model
5. Learn pickle: numpy.ndarray, png
6. ....
7. construct a DeepGP class

In [7]:
# Set random seed for reproducibility
np.random.seed(1)

# Read data
cwd = os.getcwd()
print(cwd)

X_train_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_X_train.txt")
X_test_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_X_test.txt")
Y_train_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_Y_train.txt")
Y_test_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_Y_test.txt")

x_obs = np.loadtxt(X_train_path)
y_obs = np.loadtxt(Y_train_path)
x_val = np.loadtxt(X_test_path)
y_val = np.loadtxt(Y_test_path)

# Set plot limits and labels
xlims = [-0.2, 0.2]

# The X and Y have to be at least 2-dim
x_train = torch.from_numpy(x_obs).float().reshape(-1,1)
y_train = torch.from_numpy(y_obs).float()
x_test = torch.from_numpy(x_val).float().reshape(-1,1)
y_test = torch.from_numpy(y_val).float()

C:\Users\yuanq\OneDrive\Desktopold\SB\research\DGPII\program\DGP-RFF-main


In [8]:
x_train = x_train.cuda()


In [9]:
y_train = y_train.cuda()
x_test = x_test.cuda()
y_test = y_test.cuda()

In [10]:
x_train.device

device(type='cuda', index=0)

In [11]:


  # X = [
#     [1,2,3],
#     [4,5,6,7,8]
#     ]
# X_path = os.path.join(cwd, "folds", "test.pickle")
# with open(X_path, 'wb') as f:
#     pickle.dump(X, f)

In [12]:
# with open(X_path, 'rb') as f:
#     X_read = pickle.load(f)

In [19]:
class ModelDGP(PyroModule):
    def __init__(self, dim_list=[1,1,1], J_list=[5,10]):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGP(dim_list, J_list)
        self.model.to('cuda')

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise #original 0.5 loc

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [20]:
class ModelEnsembleDGP(PyroModule):
    def __init__(self, dim_list = [1,1,1], J_list=[50,10]):
        super().__init__()
        self.out_dim = dim_list[-1]
        self.model = DeepEnsembleGP(dim_list, J_list,2)
        self.model.to('cuda')
        
    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        # mu = self.linear(x1).squeeze()
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [21]:
class ModelNoBiasDGP(PyroModule):
    def __init__(self, dim_list=[1,1,1], J_list=[50,10]):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGPNoBias(dim_list, J_list)
        self.model.to('cuda')

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        
        # batch shape | event shapt
        # 10000       |
        
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample( "obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)
            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [22]:
print([x_train.shape[1],10,y_train.shape[1]])

[1, 10, 6]


In [23]:
#model = Model(in_dim_list=[x_train.shape[1],10], out_dim_list=[10,y_train.shape[1]], J_list=[50,10])
mymodel = ModelDGP(dim_list=[x_train.shape[1],5,y_train.shape[1]], J_list=[2,3])
print(x_train.shape)
print(y_train.shape)
mymodel = mymodel.to('cuda')

[SingleGP(
  (layers): PyroModuleList(
    (0): FirstLayer(
      (layer): PyroLinear(in_features=1, out_features=4, bias=False)
    )
    (1): SecondLayer(
      (layer): PyroLinear(in_features=8, out_features=5, bias=True)
    )
  )
), SingleGP(
  (layers): PyroModuleList(
    (0): FirstLayer(
      (layer): PyroLinear(in_features=5, out_features=6, bias=False)
    )
    (1): SecondLayer(
      (layer): PyroLinear(in_features=12, out_features=6, bias=True)
    )
  )
)]
torch.Size([10000, 1])
torch.Size([10000, 6])


In [24]:
mean_field_guide = AutoDiagonalNormal(mymodel)
optimizer = pyro.optim.Adam({"lr": 0.001})

svi = SVI(mymodel, mean_field_guide, optimizer, loss=Trace_ELBO())
pyro.clear_param_store()

num_epochs = 2
progress_bar = trange(num_epochs)

for epoch in progress_bar:
    loss = svi.step(x_train, y_train)
    progress_bar.set_postfix(loss=f"{loss / x_train.shape[0]:.3f}")

  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
param_store = pyro.get_param_store()
for name, value in param_store.items():
    print(name, pyro.param(name).shape)
    print(f"{name}: {value}")

AutoDiagonalNormal.loc torch.Size([158])
AutoDiagonalNormal.loc: Parameter containing:
tensor([-0.2095,  0.7383, -0.4239, -0.1470,  4.8534,  4.9470,  4.6324,  5.3043,
         5.0025,  5.2929,  4.6596,  4.9252,  4.7462,  4.4091,  4.8242,  5.2626,
         4.9393,  5.2733,  4.2675,  5.5283,  5.2250,  5.4177,  4.7928,  5.2131,
         5.6234,  4.3138,  4.2937,  4.8516,  4.7414,  5.2840,  5.0470,  5.1272,
         4.7817,  5.0123,  5.1951,  4.5011,  4.7179,  5.2271,  5.2623,  5.5218,
         4.7710,  4.6981,  4.9352,  5.0402,  1.6104,  2.0981,  2.0443,  1.6099,
         1.8208, -0.1543, -0.5422,  0.6358,  0.5488,  0.0595,  0.5691,  0.0470,
         0.2252, -0.2822, -0.0447, -0.0849, -0.1858, -0.2934,  0.2409,  0.1322,
         0.1166, -0.1027, -0.3348,  0.1638, -0.2266, -0.0317, -0.1965,  0.0374,
        -0.2773,  0.1673,  0.2422,  0.4658, -0.2642,  0.0896, -0.1765,  5.0334,
         4.8564,  5.2522,  5.0584,  5.0216,  4.8647,  4.8684,  4.9876,  5.0167,
         4.8134,  4.6463,  4.9918

In [31]:
layer_num = 2
in_dim_list = [1,5] 
out_dim_list = [5,6]
J_list = [4,6]
len_list = []
for i in range(layer_num):
    len_list.append(J_list[i]*in_dim_list[i])
    len_list.append(out_dim_list[i]*2*J_list[i])
    len_list.append(out_dim_list[i])
len_list

[4, 40, 5, 30, 72, 6]

In [38]:
def split_array_by_lengths(array, lengths):
    result = []
    current_position = 0

    for length in lengths:
        if current_position + length <= len(array):
            result.append(array[current_position:current_position + length])
            current_position += length
        else:
            # 如果剩余的元素不足以组成一个完整的子数组，则将剩余的元素全部添加到结果中
            result.append(array[current_position:])
            break

    return result


In [39]:
loc =param_store["AutoDiagonalNormal.loc"]
loc

Parameter containing:
tensor([-0.2095,  0.7383, -0.4239, -0.1470,  4.8534,  4.9470,  4.6324,  5.3043,
         5.0025,  5.2929,  4.6596,  4.9252,  4.7462,  4.4091,  4.8242,  5.2626,
         4.9393,  5.2733,  4.2675,  5.5283,  5.2250,  5.4177,  4.7928,  5.2131,
         5.6234,  4.3138,  4.2937,  4.8516,  4.7414,  5.2840,  5.0470,  5.1272,
         4.7817,  5.0123,  5.1951,  4.5011,  4.7179,  5.2271,  5.2623,  5.5218,
         4.7710,  4.6981,  4.9352,  5.0402,  1.6104,  2.0981,  2.0443,  1.6099,
         1.8208, -0.1543, -0.5422,  0.6358,  0.5488,  0.0595,  0.5691,  0.0470,
         0.2252, -0.2822, -0.0447, -0.0849, -0.1858, -0.2934,  0.2409,  0.1322,
         0.1166, -0.1027, -0.3348,  0.1638, -0.2266, -0.0317, -0.1965,  0.0374,
        -0.2773,  0.1673,  0.2422,  0.4658, -0.2642,  0.0896, -0.1765,  5.0334,
         4.8564,  5.2522,  5.0584,  5.0216,  4.8647,  4.8684,  4.9876,  5.0167,
         4.8134,  4.6463,  4.9918,  5.2154,  4.2549,  5.1443,  4.7226,  4.3080,
         4.9549,  

In [44]:
result = split_array_by_lengths(loc, len_list)
result
# save_variable.py
# import pickle
# 
# loc_test = result
# 
# with open('loc_test.pkl', 'wb') as file:
#     pickle.dump(result, file)


[tensor([-0.2095,  0.7383, -0.4239, -0.1470], device='cuda:0',
        grad_fn=<SliceBackward0>),
 tensor([4.8534, 4.9470, 4.6324, 5.3043, 5.0025, 5.2929, 4.6596, 4.9252, 4.7462,
         4.4091, 4.8242, 5.2626, 4.9393, 5.2733, 4.2675, 5.5283, 5.2250, 5.4177,
         4.7928, 5.2131, 5.6234, 4.3138, 4.2937, 4.8516, 4.7414, 5.2840, 5.0470,
         5.1272, 4.7817, 5.0123, 5.1951, 4.5011, 4.7179, 5.2271, 5.2623, 5.5218,
         4.7710, 4.6981, 4.9352, 5.0402], device='cuda:0',
        grad_fn=<SliceBackward0>),
 tensor([1.6104, 2.0981, 2.0443, 1.6099, 1.8208], device='cuda:0',
        grad_fn=<SliceBackward0>),
 tensor([-0.1543, -0.5422,  0.6358,  0.5488,  0.0595,  0.5691,  0.0470,  0.2252,
         -0.2822, -0.0447, -0.0849, -0.1858, -0.2934,  0.2409,  0.1322,  0.1166,
         -0.1027, -0.3348,  0.1638, -0.2266, -0.0317, -0.1965,  0.0374, -0.2773,
          0.1673,  0.2422,  0.4658, -0.2642,  0.0896, -0.1765], device='cuda:0',
        grad_fn=<SliceBackward0>),
 tensor([5.0334, 4.8564

C:\Users\yuanq\OneDrive\Desktopold\SB\research\DGPII\program\DGP-RFF-main


In [ ]:
mymodel.parameters()

In [ ]:
predictive = Predictive(mymodel, guide=mean_field_guide, num_samples=500)
preds = predictive(x_test)

In [ ]:
y_pred = preds['obs'].cpu().detach().numpy().mean(axis=0)

In [ ]:
preds['obs'].shape

In [ ]:
for d in range(6):
    plt.plot(x_obs, y_obs[:,d], label="Observation")
    plt.plot(x_test.cpu(), y_pred[:,d], label="Prediction")
    plt.legend()
    plt.show()

In [ ]:
# for i in range(int(len(mymodel.model.layers))):
#     print("Layer", i,":")
#     print("RBF Omega = ", mymodel.model.layers[i].layers_RBF[0].layer.weight)
    #print("Cauchy Omega = ", model.model.layers[2*i].layers_Cauchy.omega)
    #print("Laplacian Omega = ", model.model.layers[2*i].layers_Laplacian.omega)
    #print("W = ", model.model.layers[2*i+1].layer.weight.squeeze())
    #print("bias = ", model.model.layers[2*i+1].layer.bias.squeeze())

In [ ]:
#getvalue(mymodel)

In [ ]:
# # sampled Omega: 
# for i in range(int(len(model.model.layers)/2)):
#     print("Layer", 2*i, " and Layer", 2*i+1, ":")
#     print("Omega = ", model.model.layers[2*i].layer.weight.squeeze())
#     print("W = ", model.model.layers[2*i+1].layer.weight.squeeze())
#     print("bias = ", model.model.layers[2*i+1].layer.bias.squeeze())


In [ ]:
# pickle.dump(model, open('DeepRFGP.model.pkl', 'wb'))

In [ ]:
preds['model.layers.0.layer.weight'].squeeze().mean(axis=0)#: Omega estimator

In [ ]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name).shape)

In [ ]:
mymodel.model.layers[0].layer.weight

In [ ]:
end = time.perf_counter()
runtime = end - start
print("Runtime:", runtime)

1 -> 50; 100 -> 6

Omega 1*50  1*50
W     6*100 6*100
bias  6     6
sigma 1     1

In [ ]:
# def model(data):
#   alpha = torch.tensor(6.0)
#   beta = torch.tensor(10.0)
#   pay_probs = pyro.sample('pay_probs', dist.Beta(alpha, beta).expand(3).independent(1))
#   normalized_pay_probs = pay_probs / torch.sum(pay_probs)

#   with pyro.iarange('data_loop', len(data)):
#     pyro.sample('obs', dist.Categorical(probs=normalized_pay_probs), obs=data)

In [ ]:
# def guide(data):
#   alphas = pyro.param('alphas', torch.tensor(6.).expand(3), constraint=constraints.positive)
#   betas = pyro.param('betas', torch.tensor(10.).expand(3), constraint=constraints.positive) 

#   pyro.sample('pay_probs', dist.Beta(alphas, betas).independent(1))